In [1]:
import pandas as pd
from dotenv import load_dotenv # type: ignore
import os
import sys
sys.path.append("..")
from src.get_all_files import get_all_files
from dateparser import parse
import cv2
import pytesseract
import io
from PIL import Image


In [2]:
load_dotenv()
blob_keys = os.getenv("AZURE_BLOB_KEYS")
all_files = get_all_files(blob_keys)
print(len(all_files))


5123


In [3]:
def process_image(input_img_path, output_img_path, regions):
    img = cv2.imread(input_img_path)
    extracted_texts = {}

    for region_name, (x, y, w, h) in regions.items():
        # Dessiner les rectangles et noms des régions
        top_left = (x, y)
        bottom_right = (x + w, y + h)
        cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)

        # Extraire le texte de la région
        roi = img[y:y+h, x:x+w]
        text = pytesseract.image_to_string(roi, config="--psm 6")
        extracted_texts[region_name] = text.strip()

    # Sauvegarde de l'image annotée
    cv2.imwrite(output_img_path, img)

    return extracted_texts

# Définition des blocs
predefined_regions = {
    "Adresse": (10, 116, 400, 60),
    "Nom": (70, 70, 250, 30),
    "Mail": (50, 100, 250, 20),
    "Date": (105, 45, 250, 30),
    "Products": (20, 180, 400, 350),
    "Quantities_and_prices": (540, 180, 250, 350)
}



In [4]:
df = pd.DataFrame(columns=["adress", "nom", "mail", "date", "products", "quantities", "prices", "total"])

In [7]:

file = all_files[6]
chemin = f"../data/files/{file.split('_')[1]}/{file}"
output = "../data/test.png"

# Traitement de l'image
extracted_texts = process_image(chemin, output, predefined_regions)

# Affichage des résultats
print(extracted_texts)
print(f"adresse: {extracted_texts["Adresse"].replace("\n", " ")}")
print(f"nom: {extracted_texts["Nom"]}")
print(f"mail: {extracted_texts["Mail"]}")
print(f"date: {parse(extracted_texts["Date"], languages=["fr","en"])}")
print(f"products: {[product for product in extracted_texts["Products"].split('\n') if product != "TOTAL"]}")
print(f"quantities: {[quantity.split("x")[0].strip() for quantity in extracted_texts["Quantities_and_prices"].split('\n')[:-1]]}")
print(f"prices: {[price.split("x")[1].strip().replace(" Euro", "") for price in extracted_texts["Quantities_and_prices"].split('\n')[:-1]]}")
print(f"total: {extracted_texts["Quantities_and_prices"].split('\n')[-1].replace(" Euro", "")}")


{'Adresse': 'Address 96374 Amanda Dam\nEast Craigfort, FM 73373', 'Nom': 'Bruce Pace', 'Mail': 'alan46 @example.org:', 'Date': '2018-12-01', 'Products': 'These president network actually.\nTOTAL', 'Quantities_and_prices': '3x 7.30 Euro\n21.90 Euro'}
adresse: Address 96374 Amanda Dam East Craigfort, FM 73373
nom: Bruce Pace
mail: alan46 @example.org:
date: 2018-12-01 00:00:00
products: ['These president network actually.']
quantities: ['3']
prices: ['7.30']
total: 21.90


In [43]:
# Chargement des données extraites
file = all_files[500]
chemin = f"../data/files/{file.split('_')[1]}/{file}"
output = "../data/test.png"

extracted_texts = process_image(chemin, output, predefined_regions)

# Nettoyage et formatage des données
adresse = extracted_texts["Adresse"].replace("\n", " ")
nom_client = extracted_texts["Nom"]
mail_client = extracted_texts["Mail"]
date_facturation = parse(extracted_texts["Date"], languages=["fr", "en"])

products = [product for product in extracted_texts["Products"].split('\n') if product != "TOTAL"]
quantities = [quantity.split("x")[0].strip() for quantity in extracted_texts["Quantities_and_prices"].split('\n')[:-1]]
prices = [price.split("x")[1].strip().replace(" Euro", "") for price in extracted_texts["Quantities_and_prices"].split('\n')[:-1]]
total = extracted_texts["Quantities_and_prices"].split('\n')[-1].replace(" Euro", "")

# Génération d'identifiants uniques
id_client = f"CLT_{hash(nom_client + mail_client) % 10**6}"
id_facture = file.replace(".png","")

# DataFrame Client
df_client = pd.DataFrame({
    "id_client": [id_client],
    "Nom": [nom_client],
    "mail": [mail_client],
    "birthday": [None]  # Date de naissance inconnue ici
})

# DataFrame Facture
df_facture = pd.DataFrame({
    "id_facture": [id_facture],
    "texte": [adresse],
    "date_facturation": [date_facturation]
})

# DataFrame Produit
df_produit = pd.DataFrame({
    "Id_produit": [f"PROD_{hash(p) % 10**6}" for p in products],
    "Nom": products,
    "Prix": prices
})

# DataFrame Achat
df_achat = pd.DataFrame({
    "Id_produit": [f"PROD_{hash(p) % 10**6}" for p in products],
    "id_client": [id_client] * len(products),
    "id_facture": [id_facture] * len(products),
    "quantité": quantities
})

# Affichage des DataFrames
print("Clients:\n") 
display(df_client)
print("Factures:\n")
display(df_facture)
print("Produits:\n")
display(df_produit)
print("Achats:\n")
display(df_achat)


Clients:



,id_client,Nom,mail,birthday
0,CLT_410183,,cindy? 2@example.com,None


Factures:



,id_facture,texte,date_facturation
0,FAC_2019_0453-218,Address Unit 9777 Box 0560 DPO AP 29783,None


Produits:



,Id_produit,Nom,Prix
0,PROD_998674,Health year especially she.,123.28
1,PROD_301269,Evidence perform get according.,13.31


Achats:



,Id_produit,id_client,id_facture,quantité
0,PROD_998674,CLT_410183,FAC_2019_0453-218,3
1,PROD_301269,CLT_410183,FAC_2019_0453-218,1


In [6]:

df_clients = pd.DataFrame(columns=["id_client", "Nom", "mail", "birthday"])
df_factures = pd.DataFrame(columns=["id_facture", "texte", "date_facturation"])
df_produits = pd.DataFrame(columns=["Id_produit", "Nom", "Prix"])
df_achats = pd.DataFrame(columns=["Id_produit", "id_client", "id_facture", "quantité"])

def nettoyer_total(total):
    """Vérifie si le total est bien détecté (ne doit pas contenir 'x')"""
    if "x" in total:
        return None  # Erreur de détection
    return total.replace(" Euro", "")

def extraire_donnees(file):
    """Extrait et nettoie les données d'un fichier"""
    chemin = f"../data/files/{file.split('_')[1]}/{file}"
    output = "../data/test.png"

    try:
        extracted_texts = process_image(chemin, output, predefined_regions)
        
        # Nettoyage et formatage
        adresse = extracted_texts["Adresse"].replace("\n", " ")
        nom_client = extracted_texts["Nom"]
        mail_client = extracted_texts["Mail"]
        date_facturation = parse(extracted_texts["Date"], languages=["fr", "en"])

        products = [product for product in extracted_texts["Products"].split('\n') if product != "TOTAL"]
        quantities = [quantity.split("x")[0].strip() for quantity in extracted_texts["Quantities_and_prices"].split('\n')[:-1]]
        prices = [price.split("x")[1].strip().replace(" Euro", "") for price in extracted_texts["Quantities_and_prices"].split('\n')[:-1]]
        total = nettoyer_total(extracted_texts["Quantities_and_prices"].split('\n')[-1])

        # Vérification des erreurs
        erreurs = []
        if not nom_client: erreurs.append("Nom non détecté")
        if not mail_client: erreurs.append("Mail non détecté")
        if not date_facturation: erreurs.append("Date non détectée")
        if not products: erreurs.append("Produits non détectés")
        if not quantities: erreurs.append("Quantités non détectées")
        if not prices: erreurs.append("Prix non détectés")
        if total is None: erreurs.append("Total mal détecté")

        if erreurs:
            print(f"Erreur dans le fichier {file} : {', '.join(erreurs)}")
            return None  # On saute ce fichier en cas d'erreur

        # Génération d'identifiants uniques
        id_client = f"CLT_{hash(nom_client + mail_client) % 10**6}"
        id_facture = f"FACT_{hash(file) % 10**6}"

        # Création des DataFrames temporaires
        df_client = pd.DataFrame([{
            "id_client": id_client,
            "Nom": nom_client,
            "mail": mail_client,
            "birthday": None
        }])

        df_facture = pd.DataFrame([{
            "id_facture": id_facture,
            "texte": adresse,
            "date_facturation": date_facturation
        }])

        df_produit = pd.DataFrame({
            "Id_produit": [f"PROD_{hash(p) % 10**6}" for p in products],
            "Nom": products,
            "Prix": prices
        })

        df_achat = pd.DataFrame({
            "Id_produit": [f"PROD_{hash(p) % 10**6}" for p in products],
            "id_client": [id_client] * len(products),
            "id_facture": [id_facture] * len(products),
            "quantité": quantities
        })

        return df_client, df_facture, df_produit, df_achat

    except Exception as e:
        print(f"Échec pour le fichier {file} : {str(e)}")
        return None

# Boucle sur les 500 premiers fichiers
for file in all_files[:500]:
    data = extraire_donnees(file)
    
    if data:
        df_client, df_facture, df_produit, df_achat = data

        # Ajout aux DataFrames globaux
        df_clients = pd.concat([df_clients, df_client], ignore_index=True)
        df_factures = pd.concat([df_factures, df_facture], ignore_index=True)
        df_produits = pd.concat([df_produits, df_produit], ignore_index=True)
        df_achats = pd.concat([df_achats, df_achat], ignore_index=True)

# Affichage final des DataFrames après traitement
print("\n✅ Traitement terminé. Voici les DataFrames finaux :")

print("Clients :")
display(df_clients)

print("Factures :")
display(df_factures)

print("Produits :")
display(df_produits)

print("Achats :")
display(df_achats)


Échec pour le fichier FAC_2018_0001-654.png : list index out of range


C:\Users\melos\AppData\Local\Temp\ipykernel_23172\3984375533.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_factures = pd.concat([df_factures, df_facture], ignore_index=True)


Erreur dans le fichier FAC_2018_0003-025.png : Total mal détecté
Échec pour le fichier FAC_2018_0004-759.png : All arrays must be of the same length
Erreur dans le fichier FAC_2018_0005-281.png : Date non détectée
Échec pour le fichier FAC_2018_0006-250.png : list index out of range
Erreur dans le fichier FAC_2018_0009-754.png : Nom non détecté
Erreur dans le fichier FAC_2018_0010-104.png : Nom non détecté
Échec pour le fichier FAC_2018_0011-692.png : All arrays must be of the same length
Échec pour le fichier FAC_2018_0012-758.png : list index out of range
Échec pour le fichier FAC_2018_0013-913.png : All arrays must be of the same length
Échec pour le fichier FAC_2018_0014-558.png : All arrays must be of the same length
Erreur dans le fichier FAC_2018_0015-089.png : Total mal détecté
Échec pour le fichier FAC_2018_0016-604.png : list index out of range
Erreur dans le fichier FAC_2018_0017-432.png : Nom non détecté, Date non détectée
Erreur dans le fichier FAC_2018_0018-032.png : Date

,id_client,Nom,mail,birthday
0,CLT_234849,Samuel Coleman,gmeyer@example.com,None
1,CLT_235337,Bruce Pace,alan46 @example.org:,None
2,CLT_173711,David Macdonald,zobnen@example.com,None
3,CLT_587979,Mario Benson,julle45@example.org,None
4,CLT_756709,Melissa Moreno.,"xdunn@example.org,",None
...,...,...,...,...
137,CLT_552366,Sarah Burgess MD,gpena@example.com,None
138,CLT_173757,Raymond Waters,Jimmyparks@example.com,None
139,CLT_573954,‘Angela Nguyen.,moralesjeanne@example.com,None
140,CLT_258813,James Nguyen,tprice@example.com,None


Factures :


,id_facture,texte,date_facturation
0,FACT_801945,"Address 64623 Wright Mills Turnermouth, KS 45555",2018-10-17
1,FACT_683158,"Address 96374 Amanda Dam East Craigfort, FM 73373",2018-12-01
2,FACT_103513,Address 720 Norman Stravenue Apt. 861 Maysfort...,2018-12-03
3,FACT_328312,"Address PSC 1953, Bax 3013 APO AP 95181",2018-12-14
4,FACT_123228,Address 4263 Yoder Viaduct Suite 458 North Deb...,2018-12-16
...,...,...,...
137,FACT_147702,Address 07718 Elizabeth Stream Suite 663 East ...,2019-07-05
138,FACT_615672,Address 1938 James Flats Suite 901 Port Kennet...,2019-07-06
139,FACT_558623,Address 7648 Payne Manor Suite 778 ‘Trevorport...,2019-07-07
140,FACT_905198,"Address 6544 Lopez Flat New Kevin, WI00743",2019-07-09


Produits :


,Id_produit,Nom,Prix
0,PROD_252511,Between everybody size conference.,45.70
1,PROD_933588,These president network actually.,7.30
2,PROD_272088,Statement sort where on.,21.15
3,PROD_543504,Clearly treatment for up.,14.49
4,PROD_441413,Brother pretty local likely.,78.32
...,...,...,...
253,PROD_413918,Interview actually quite bill.,89.34
254,PROD_867235,Professional of indeed security.,24.85
255,PROD_134754,Likely million school education.,18.66
256,PROD_304697,Business pattern should popular.,23.20


Achats :


,Id_produit,id_client,id_facture,quantité
0,PROD_252511,CLT_234849,FACT_801945,4
1,PROD_933588,CLT_235337,FACT_683158,3
2,PROD_272088,CLT_173711,FACT_103513,1
3,PROD_543504,CLT_173711,FACT_103513,1
4,PROD_441413,CLT_173711,FACT_103513,3
...,...,...,...,...
253,PROD_413918,CLT_258813,FACT_905198,4
254,PROD_867235,CLT_258813,FACT_905198,1
255,PROD_134754,CLT_258813,FACT_905198,4
256,PROD_304697,CLT_258813,FACT_905198,1


Beaucoup trop d'échec, seulement 140 / 500 correctement lus, en 9 minutes

# Avec Azure

In [57]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import time

# Configuration du client
endpoint = "https://francecentral.api.cognitive.microsoft.com/"
key = "5b3903aa12104b8c9e036e01c9ef6f80"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(key))

# Chemin de l'image locale
file = all_files[4499]
chemin = f"../data/files/{file.split('_')[1]}/{file}"
local_image_path = chemin

# Ouvrir l'image
with open(local_image_path, "rb") as image_stream:
    # Appeler l'API OCR
    read_response = computervision_client.read_in_stream(image_stream, raw=True)

    # Récupérer l'ID de l'opération pour vérifier le résultat
    read_operation_location = read_response.headers["Operation-Location"]
    operation_id = read_operation_location.split("/")[-1]

    # Attendre que l'analyse soit terminée
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
            break
        time.sleep(1)

    # Afficher le texte extrait
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                print(line.text)


INVOICE FAC/2024/0220
Issue date 2024-04-11
Bill to Deborah Phillips
Email hardymaurice@example.net
Brilllling
Address 3931 Anthony Locks Apt. 747
Greerborough, WI 45662
Read create no office.
2 x 119.10 Euro
Themselves individual identify scene.
4 x 134.12 Euro
TOTAL
774.68 Euro


In [4]:
import io
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
from PIL import Image, ImageOps
import io

from PIL import Image, ImageOps
import io

def add_white_border_to_region(region, img):
    x, y, w, h = region
    img_width, img_height = img.size  # Dimensions de l'image originale

    # Créer une image de fond blanc de la même taille que l'image d'origine
    new_img = Image.new("RGB", (img_width, img_height), (255, 255, 255))  # Image avec fond blanc

    # Recadrer la région d'origine
    region_img = img.crop((x, y, x + w, y + h))

    # Coller la région recadrée au centre de l'image avec fond blanc
    new_x = x
    new_y = y
    new_img.paste(region_img, (new_x, new_y))

    return new_img

def process_image_azure(input_img_path, regions, endpoint, key):
    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(key))
    extracted_texts = {}

    try:
        with Image.open(input_img_path) as img:
            img_width, img_height = img.size  # Dimensions de l'image

            for region_name, (x, y, w, h) in regions.items():
                # Ajouter du blanc autour de la région sans changer ses dimensions
                new_img = add_white_border_to_region((x, y, w, h), img)

                # Convertir l'image modifiée en bytes pour l'API OCR
                img_byte_arr = io.BytesIO()
                new_img.save(img_byte_arr, format='PNG')
                img_byte_arr.seek(0)

                # Appel à l'API OCR Azure
                try:
                    read_response = computervision_client.read_in_stream(img_byte_arr, raw=True)
                    operation_location = read_response.headers["Operation-Location"]
                    operation_id = operation_location.split("/")[-1]

                    # Attendre que l'analyse soit terminée
                    while True:
                        read_result = computervision_client.get_read_result(operation_id)
                        if read_result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
                            break
                        time.sleep(1)

                    # Extraire le texte si l'analyse est réussie
                    if read_result.status == OperationStatusCodes.succeeded:
                        text = "\n".join(line.text for text_result in read_result.analyze_result.read_results for line in text_result.lines)
                        extracted_texts[region_name] = text.strip()
                    else:
                        print(f"Erreur OCR pour la région '{region_name}': {read_result.status}")
                except Exception as e:
                    print(f"Erreur lors de l'appel à l'API OCR pour la région '{region_name}': {e}")

    except Exception as e:
        print(f"Erreur lors de l'ouverture ou du traitement de l'image {input_img_path}: {e}")

    return extracted_texts


# Définition des régions
predefined_regions = {
    "Adresse": (10, 116, 400, 60),
    "Nom": (70, 70, 250, 30),
    "Mail": (55, 100, 250, 20),
    "Date": (105, 45, 250, 30),
    "Products": (20, 180, 400, 350),
    "Quantities_and_prices": (540, 180, 250, 350)
}

# Configuration Azure
endpoint = "https://francecentral.api.cognitive.microsoft.com/"
key = "5b3903aa12104b8c9e036e01c9ef6f80"

# Vérifier que la variable all_files existe
try:
    file = all_files[5000]
    chemin = f"../data/files/{file.split('_')[1]}/{file}"
    extracted_texts = process_image_azure(chemin, predefined_regions, endpoint, key)

    print(f"adresse: {extracted_texts.get('Adresse', '').replace('\n', ' ')}")
    print(f"nom: {extracted_texts.get('Nom', '')}")
    print(f"mail: {extracted_texts.get('Mail', '')}")
    print(f"date: {extracted_texts.get('Date', '')}")
    print(f"products: {[product for product in extracted_texts.get('Products', '').split('\n') if product != 'TOTAL']}")
    print(f"quantities_and_prices: {extracted_texts.get('Quantities_and_prices', '')}")
except NameError:
    print("Erreur : la variable all_files n'est pas définie.")


adresse: Address 3887 Haynes Circle Apt. 995 West Sallyville, WV 87676
nom: Gary Carrillo
mail: meredithturner@example.com
date: 2024-11-06
products: ['Source edge score their.', 'Teach themselves despite we.', 'Especially environmental through spring.', 'Phone interesting a look.', 'Cost from without stage.']
quantities_and_prices: 2 x
25.04 Euro
4 x
34.70 Euro
1 x
12.72 Euro
1 x
43.49 Euro
4 x
3.31 Euro
258.33 Euro


Plus efficace mais bien plus lent, trop lent pour utiliser la technique de faire par bout d'image